# FINAL PROJECT CODE

## The first step is to import every library we will need to perform the analysis

In [1]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option("display.max_columns", None)
pd.set_option("display.max_rows", None)

import json # library to handle JSON files

from geopy.geocoders import Nominatim # convert an address into latitude and longitude values
!pip install geocoder
import geocoder # to get coordinates

import requests # library to handle requests
from bs4 import BeautifulSoup # library to parse HTML and XML documents

from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

!pip install folium
import folium # map rendering library

/opt/conda/envs/Python-3.7-main/lib/python3.7/site-packages/secretstorage/dhcrypto.py:16: CryptographyDeprecationWarning: int_from_bytes is deprecated, use int.from_bytes instead
  from cryptography.utils import int_from_bytes
/opt/conda/envs/Python-3.7-main/lib/python3.7/site-packages/secretstorage/util.py:25: CryptographyDeprecationWarning: int_from_bytes is deprecated, use int.from_bytes instead
  from cryptography.utils import int_from_bytes
     |████████████████████████████████| 98 kB 11.6 MB/s eta 0:00:01
/opt/conda/envs/Python-3.7-main/lib/python3.7/site-packages/secretstorage/dhcrypto.py:16: CryptographyDeprecationWarning: int_from_bytes is deprecated, use int.from_bytes instead
  from cryptography.utils import int_from_bytes
/opt/conda/envs/Python-3.7-main/lib/python3.7/site-packages/secretstorage/util.py:25: CryptographyDeprecationWarning: int_from_bytes is deprecated, use int.from_bytes instead
  from cryptography.utils import int_from_bytes
     |██████████████████████████

### Scraping data from the wikipedia web to gather information about neighborhoods in Madrid

In [2]:
data = requests.get("https://en.wikipedia.org/wiki/List_of_neighborhoods_of_Madrid").text
soup = BeautifulSoup(data, 'lxml')
textList = []
neighborhoodList = []
neighborhoodList.clear()


In [3]:
neighborhoods = ["Arganzuela","Barajas","Carabanchel","Centro","Chamartín","Chamberí","Ciudad Lineal","Fuencarral - El Pardo","Hortaleza","Latina","Moncloa - Aravaca","Moratalaz","Puente de Vallecas","Retiro","Salamanca","San Blas - Canillejas","Tetuán","Usera","Vicálvaro","Villa de Vallecas","Villaverde","Palacio","Embajadores","Cortes","Justicia","Universidad","Sol","Imperial","Acacias","Chopera","Legazpi","Delicias","Palos de Moguer","Atocha","Pacifico","Adelfas","Estrella","Ibiza","Los Jerónimos","Niño Jesús","Recoletos","Goya","Fuente del Berro","Guindalera","Lista","Castellana","El Viso","Prosperidad","Ciudad Jardin","Hispanoamérica","Nueva España","Castilla","Bellas Vistas","Cuatro Caminos","Castillejos","Almenara","Valdeacederas","Berruguete","Gaztambide","Arapiles","Trafalgar","Almagro","Ríos Rosas"]


In [4]:
#for row in soup.find_all("tr")[0].find_all("tr"):
for row in soup.find_all('table')[0].tbody.find_all('tr'):
    first_column = row.find_all('th')
    neighborhoodList.append(first_column)
    
df = pd.DataFrame({"Neighborhood": neighborhoodList})
df1 = df.iloc[1:]
madrid_df = df1.reset_index(drop=True)
madrid_df.head()

neighborhoodsList = pd.DataFrame({"Neighborhood":neighborhoods})
neighborhoodsList.head(15)

,Neighborhood
0,Arganzuela
1,Barajas
2,Carabanchel
3,Centro
4,Chamartín
5,Chamberí
6,Ciudad Lineal
7,Fuencarral - El Pardo
8,Hortaleza
9,Latina


### We now need the specific location (latitude, longitude) of each neighborhood in Madrid

In [5]:
# Geographical coordinates of neighborhoods

# define a function to get coordinates
def get_latlng(neighborhood):
    # initialize your variable to None
    lat_lng_coords = None
    # loop until you get the coordinates
    while(lat_lng_coords is None):
        g = geocoder.arcgis('{}, Madrid, Spain'.format(neighborhood))
        lat_lng_coords = g.latlng
    return lat_lng_coords

coords = [ get_latlng(neighborhood) for neighborhood in neighborhoodsList["Neighborhood"].tolist() ]

df_coords = pd.DataFrame(coords, columns=['Latitude', 'Longitude'])

# merge the coordinates into the original dataframe
neighborhoodsList['Latitude'] = df_coords['Latitude']
neighborhoodsList['Longitude'] = df_coords['Longitude']


# check the neighborhoods and the coordinates
print(neighborhoodsList.shape)
neighborhoodsList.head(15)

(63, 3)


,Neighborhood,Latitude,Longitude
0,Arganzuela,40.40021,-3.69618
1,Barajas,40.49181,-3.56948
2,Carabanchel,40.39094,-3.72420
3,Centro,41.62812,-4.72705
4,Chamartín,40.45000,-3.70000
5,Chamberí,40.43404,-3.70379
6,Ciudad Lineal,40.45349,-3.65434
7,Fuencarral - El Pardo,40.49840,-3.73140
8,Hortaleza,40.47444,-3.64110
9,Latina,40.38897,-3.74569


### We save the dataframe as a CSV file

In [6]:
neighborhoodsList.to_csv("madrid_neighborhoods.csv", index=False)

### Now we proceed to obtain the exact coordinates of the city of Madrid to create the map about neighborhoods in Madrid

In [7]:
geolocator = Nominatim(user_agent="http")
location = geolocator.geocode('Madrid, Spain')
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Madrid is {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Madrid is 40.4167047, -3.7035825.


In [8]:
# create map of Berlin using latitude and longitude values
map_madrid = folium.Map(location=[latitude, longitude], zoom_start=11)

# add markers to map
for lat, lng, neighborhood in zip(neighborhoodsList['Latitude'], neighborhoodsList['Longitude'], neighborhoodsList['Neighborhood']):
    label = '{}'.format(neighborhood)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7).add_to(map_madrid)  
    
map_madrid

### Having obtain the coordinates of each neighborhood, it is now time to work with Foursquare and search for the venues we want to study. We will also calculate the coordinates of the venues.

In [9]:
CLIENT_ID = "ECUGWIPDIDJMU55URTU5W0BSUTVMVA0ZR4ZIYJVED0PUQKHM"
CLIENT_SECRET = "CWKKZIBB0UF15BVG2S1DEU43NS0K1L4ZV3QGYFLH0DLF44DR"
VERSION = "20210419"

In [10]:
radius = 2000
LIMIT = 100

venues = []
for lat, long, neighborhood in zip(neighborhoodsList['Latitude'], neighborhoodsList['Longitude'], neighborhoodsList['Neighborhood']):
    
    # create the API request URL
    url = "https://api.foursquare.com/v2/venues/explore?client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}".format(
        CLIENT_ID,
        CLIENT_SECRET,
        VERSION,
        lat,
        long,
        radius, 
        LIMIT)
    
    # make the GET request
    results = requests.get(url).json()["response"]['groups'][0]['items']
    
    # return only relevant information for each nearby venue
    for venue in results:
        venues.append((
            neighborhood,
            lat, 
            long, 
            venue['venue']['name'], 
            venue['venue']['location']['lat'], 
            venue['venue']['location']['lng'],  
            venue['venue']['categories'][0]['name']))

In [11]:
# convert the venues list into a new DataFrame
venues_df = pd.DataFrame(venues)

# define the column names
venues_df.columns = ['Neighborhood', 'Latitude', 'Longitude', 'VenueName', 'VenueLatitude', 'VenueLongitude', 'VenueCategory']

print(venues_df.shape)
venues_df.head(15)

(5870, 7)


,Neighborhood,Latitude,Longitude,VenueName,VenueLatitude,VenueLongitude,VenueCategory
0,Arganzuela,40.40021,-3.69618,Havana Blues,40.402050,-3.698488,Cuban Restaurant
1,Arganzuela,40.40021,-3.69618,Mercado de Motores,40.399149,-3.691978,Flea Market
2,Arganzuela,40.40021,-3.69618,Tres Cerditos,40.397316,-3.694184,Chinese Restaurant
3,Arganzuela,40.40021,-3.69618,Salón de Té Al Yabal,40.399015,-3.700249,Cocktail Bar
4,Arganzuela,40.40021,-3.69618,Museo del Ferrocarril (Antigua Estación de Del...,40.399395,-3.692286,Museum
5,Arganzuela,40.40021,-3.69618,Pui Thai Tapas,40.404475,-3.696692,Thai Restaurant
6,Arganzuela,40.40021,-3.69618,Mercado de Santa María de la Cabeza,40.402860,-3.696819,Farmers Market
7,Arganzuela,40.40021,-3.69618,Cervecissimus Delicias,40.404211,-3.694658,Beer Bar
8,Arganzuela,40.40021,-3.69618,Bodegas Rosell,40.403803,-3.690620,Spanish Restaurant
9,Arganzuela,40.40021,-3.69618,Magasand Deli,40.396811,-3.691293,Restaurant


In [12]:
venues_df.groupby(["Neighborhood"]).count()
print('There are {} uniques categories.'.format(len(venues_df['VenueCategory'].unique())))

There are 224 uniques categories.


### With the venues and the coordinates of each neighborhood, we calculate the proportion of venues per neighborhood

In [13]:
# one hot encoding
onehot = pd.get_dummies(venues_df[['VenueCategory']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
onehot['Neighborhoods'] = venues_df['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [onehot.columns[-1]] + list(onehot.columns[:-1])
onehot = onehot[fixed_columns]

print(onehot.shape)

grouped = onehot.groupby(["Neighborhoods"]).mean().reset_index()

print(grouped.shape)
grouped.head(15)

(5870, 225)
(63, 225)


,Neighborhoods,Accessories Store,Airport,Airport Lounge,Airport Service,American Restaurant,Aquarium,Argentinian Restaurant,Art Gallery,Art Museum,Art Studio,Asian Restaurant,Athletics & Sports,BBQ Joint,Bakery,Bank,Bar,Basketball Court,Basketball Stadium,Bed & Breakfast,Beer Bar,Beer Garden,Beer Store,Big Box Store,Bistro,Board Shop,Bookstore,Boutique,Boxing Gym,Brazilian Restaurant,Breakfast Spot,Brewery,Bridge,Bubble Tea Shop,Building,Burger Joint,Burrito Place,Bus Station,Cafeteria,Café,Candy Store,Cheese Shop,Chinese Restaurant,Chocolate Shop,Church,Circus,City Hall,Clothing Store,Cocktail Bar,Coffee Shop,Comedy Club,Comfort Food Restaurant,Comic Shop,Concert Hall,Construction & Landscaping,Convenience Store,Cosmetics Shop,Cuban Restaurant,Cupcake Shop,Deli / Bodega,Department Store,Dessert Shop,Diner,Discount Store,Dive Shop,Dog Run,Donut Shop,Drive-in Theater,Dumpling Restaurant,Duty-free Shop,Eastern European Restaurant,Electronics Store,Embassy / Consulate,Event Space,Exhibit,Fabric Shop,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Fish Market,Flea Market,Food & Drink Shop,Food Court,Food Service,Fountain,Frozen Yogurt Shop,Furniture / Home Store,Garden,Gastropub,Gay Bar,General Entertainment,German Restaurant,Gift Shop,Golf Course,Gourmet Shop,Greek Restaurant,Grocery Store,Gun Range,Gym,Gym / Fitness Center,Gym Pool,Gymnastics Gym,Health & Beauty Service,Historic Site,History Museum,Hostel,Hotel,Hotel Bar,IT Services,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indie Theater,Irish Pub,Italian Restaurant,Japanese Restaurant,Jewelry Store,Juice Bar,Kebab Restaurant,Korean Restaurant,Latin American Restaurant,Library,Liquor Store,Lounge,Market,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Modern European Restaurant,Monument / Landmark,Motorcycle Shop,Mountain,Movie Theater,Multiplex,Museum,Music Venue,Neighborhood,New American Restaurant,Nightclub,Noodle House,North Indian Restaurant,Office,Opera House,Other Nightlife,Outdoors & Recreation,Paella Restaurant,Palace,Paper / Office Supplies Store,Park,Pastry Shop,Performing Arts Venue,Persian Restaurant,Peruvian Restaurant,Pet Store,Pharmacy,Pie Shop,Pilates Studio,Pizza Place,Planetarium,Playground,Plaza,Police Station,Polish Restaurant,Pool,Pool Hall,Pub,Public Art,Racetrack,Ramen Restaurant,Residential Building (Apartment / Condo),Restaurant,Road,Rock Club,Roof Deck,Salad Place,Salon / Barbershop,Sandwich Place,Scandinavian Restaurant,Scenic Lookout,Science Museum,Sculpture Garden,Seafood Restaurant,Shoe Store,Shopping Mall,Skate Park,Skating Rink,Snack Place,Soccer Field,Soccer Stadium,Soup Place,Souvlaki Shop,Spa,Spanish Restaurant,Sporting Goods Shop,Sports Bar,Sports Club,Stadium,Steakhouse,Supermarket,Sushi Restaurant,Tapas Restaurant,Tea Room,Tennis Court,Tennis Stadium,Thai Restaurant,Theater,Theme Restaurant,Toy / Game Store,Trail,Train Station,Turkish Restaurant,Used Bookstore,Vegetarian / Vegan Restaurant,Venezuelan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Wine Shop,Women's Store,Yoga Studio,Zoo
0,Acacias,0.000000,0.000000,0.000000,0.000000,0.01,0.0,0.01,0.04,0.02,0.00,0.00,0.00,0.000000,0.010000,0.000000,0.060000,0.0,0.0,0.0,0.01,0.000000,0.00,0.000000,0.01,0.0,0.00,0.00,0.0,0.00,0.020000,0.010000,0.010000,0.0,0.00,0.000000,0.00,0.000000,0.000000,0.030000,0.01,0.00,0.01,0.0,0.01,0.01,0.0,0.000000,0.010000,0.040000,0.000000,0.00,0.00,0.00,0.0,0.00,0.000000,0.01,0.00,0.000000,0.000000,0.000000,0.01,0.0,0.0,0.0,0.0,0.00,0.00,0.000000,0.00,0.000000,0.00,0.01,0.00,0.0,0.0,0.01,0.000000,0.00,0.02,0.01,0.0,0.0,0.00,0.0,0.00,0.01,0.000000,0.01,0.000000,0.0,0.00,0.0,0.000000,0.00,0.000000,0.0,0.000000,0.010000,0.0,0.01,0.00,0.0,0.01,0.02,0.000000,0.00,0.0,0.020000,0.00,0.02,0.010000,0.0,0.020000,0.000000,0.00,0.000000,0.00,0.00,0.0,0.00,0.01,0.0,0.03,0.020000,0.0,0.0,0.00,0.0,0.00,0.00,0.000000,0.01,0.00,0.0,0.010000,0.00,0.01,0.00,0.00,0.00,0.00

### Proportion of restaurants in Madrid

In [14]:
d=grouped["American Restaurant"]+grouped["Argentinian Restaurant"]+grouped["Asian Restaurant"]+grouped["BBQ Joint"]+grouped["Brazilian Restaurant"]+grouped["Burrito Place"]+grouped["Chinese Restaurant"]+grouped["Diner"]+grouped["Dumpling Restaurant"]+grouped["Eastern European Restaurant"]+grouped["Falafel Restaurant"]+grouped["Fast Food Restaurant"]+grouped["Greek Restaurant"]+grouped["Indian Restaurant"]+grouped["Italian Restaurant"]+grouped["Japanese Restaurant"]+grouped["Kebab Restaurant"]+grouped["Korean Restaurant"]+grouped["Latin American Restaurant"]+grouped["Mediterranean Restaurant"]+grouped["Mexican Restaurant"]+grouped["Modern European Restaurant"]+grouped["Persian Restaurant"]+grouped["Peruvian Restaurant"]+grouped["Restaurant"]+grouped["Scandinavian Restaurant"]+grouped["Seafood Restaurant"]+grouped["Spanish Restaurant"]+grouped["Steakhouse"]+grouped["Sushi Restaurant"]+grouped["Tapas Restaurant"]+grouped["Thai Restaurant"]+grouped["Vegetarian / Vegan Restaurant"]+grouped["Vietnamese Restaurant"]+grouped["Burger Joint"]+grouped["Burrito Place"]+grouped["Cafeteria"]+grouped["Comfort Food Restaurant"]+grouped["Gastropub"]+grouped["North Indian Restaurant"]+grouped["Juice Bar"]+grouped["Paella Restaurant"]+grouped["Pizza Place"]+grouped["Polish Restaurant"]+grouped["Salad Place"]+grouped["Venezuelan Restaurant"]+grouped["Wine Bar"]+grouped["Tea Room"]
d.sum()

27.26622372889604

### Proportion of cultural spaces in Madrid

In [15]:
g=grouped["Art Gallery"]+grouped["Art Museum"]+grouped["Concert Hall"]+grouped["General Entertainment"]+grouped["History Museum"]+grouped["Library"]+grouped["Monument / Landmark"]+grouped["Museum"]+grouped["Movie Theater"]+grouped["Music Venue"]+grouped["Performing Arts Venue"]+grouped["Science Museum"]+grouped["Sculpture Garden"]+grouped["Stadium"]+grouped["Theater"]
g.sum()

4.179598997493734

### Proportion of hotels in Madrid

In [16]:
h=grouped["Bed & Breakfast"]+grouped["Hostel"]+grouped["Hotel"]+grouped["Hotel Bar"]
h.sum()

2.3282726045883937

### Proportion of green spots in Madrid

In [17]:
j=grouped["Aquarium"]+grouped["Bridge"]+grouped["Fountain"]+grouped["Garden"]+grouped["Mountain"]+grouped["Park"]+grouped["Playground"]+grouped["Pool"]+grouped["Scenic Lookout"]+grouped["Sculpture Garden"]
j.sum()

2.9434313566710038

### Proportion of shops in Madrid

In [18]:
k=grouped["Accessories Store"]+grouped["Bakery"]+grouped["Beer Store"]+grouped["Big Box Store"]+grouped["Boutique"]+grouped["Bubble Tea Shop"]+grouped["Candy Store"]+grouped["Cheese Shop"]+grouped["Chocolate Shop"]+grouped["Clothing Store"]+grouped["Coffee Shop"]+grouped["Convenience Store"]+grouped["Cosmetics Shop"]+grouped["Cupcake Shop"]+grouped["Dessert Shop"]+grouped["Discount Store"]+grouped["Donut Shop"]+grouped["Electronics Store"]+grouped["Fabric Shop"]+grouped["Food & Drink Shop"]+grouped["Furniture / Home Store"]+grouped["Gift Shop"]+grouped["Gourmet Shop"]+grouped["Grocery Store"]+grouped["Ice Cream Shop"]+grouped["Liquor Store"]+grouped["Miscellaneous Shop"]+grouped["Mobile Phone Shop"]+grouped["Motorcycle Shop"]+grouped["Pastry Shop"]+grouped["Pie Shop"]+grouped["Pet Store"]+grouped["Shopping Mall"]+grouped["Toy / Game Store"]+grouped["Wine Shop"]
k.sum()

7.7770748499149285

### General description of the results obtained

In [19]:
grouped.describe()

,Accessories Store,Airport,Airport Lounge,Airport Service,American Restaurant,Aquarium,Argentinian Restaurant,Art Gallery,Art Museum,Art Studio,Asian Restaurant,Athletics & Sports,BBQ Joint,Bakery,Bank,Bar,Basketball Court,Basketball Stadium,Bed & Breakfast,Beer Bar,Beer Garden,Beer Store,Big Box Store,Bistro,Board Shop,Bookstore,Boutique,Boxing Gym,Brazilian Restaurant,Breakfast Spot,Brewery,Bridge,Bubble Tea Shop,Building,Burger Joint,Burrito Place,Bus Station,Cafeteria,Café,Candy Store,Cheese Shop,Chinese Restaurant,Chocolate Shop,Church,Circus,City Hall,Clothing Store,Cocktail Bar,Coffee Shop,Comedy Club,Comfort Food Restaurant,Comic Shop,Concert Hall,Construction & Landscaping,Convenience Store,Cosmetics Shop,Cuban Restaurant,Cupcake Shop,Deli / Bodega,Department Store,Dessert Shop,Diner,Discount Store,Dive Shop,Dog Run,Donut Shop,Drive-in Theater,Dumpling Restaurant,Duty-free Shop,Eastern European Restaurant,Electronics Store,Embassy / Consulate,Event Space,Exhibit,Fabric Shop,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Fish Market,Flea Market,Food & Drink Shop,Food Court,Food Service,Fountain,Frozen Yogurt Shop,Furniture / Home Store,Garden,Gastropub,Gay Bar,General Entertainment,German Restaurant,Gift Shop,Golf Course,Gourmet Shop,Greek Restaurant,Grocery Store,Gun Range,Gym,Gym / Fitness Center,Gym Pool,Gymnastics Gym,Health & Beauty Service,Historic Site,History Museum,Hostel,Hotel,Hotel Bar,IT Services,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indie Theater,Irish Pub,Italian Restaurant,Japanese Restaurant,Jewelry Store,Juice Bar,Kebab Restaurant,Korean Restaurant,Latin American Restaurant,Library,Liquor Store,Lounge,Market,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Modern European Restaurant,Monument / Landmark,Motorcycle Shop,Mountain,Movie Theater,Multiplex,Museum,Music Venue,Neighborhood,New American Restaurant,Nightclub,Noodle House,North Indian Restaurant,Office,Opera House,Other Nightlife,Outdoors & Recreation,Paella Restaurant,Palace,Paper / Office Supplies Store,Park,Pastry Shop,Performing Arts Venue,Persian Restaurant,Peruvian Restaurant,Pet Store,Pharmacy,Pie Shop,Pilates Studio,Pizza Place,Planetarium,Playground,Plaza,Police Station,Polish Restaurant,Pool,Pool Hall,Pub,Public Art,Racetrack,Ramen Restaurant,Residential Building (Apartment / Condo),Restaurant,Road,Rock Club,Roof Deck,Salad Place,Salon / Barbershop,Sandwich Place,Scandinavian Restaurant,Scenic Lookout,Science Museum,Sculpture Garden,Seafood Restaurant,Shoe Store,Shopping Mall,Skate Park,Skating Rink,Snack Place,Soccer Field,Soccer Stadium,Soup Place,Souvlaki Shop,Spa,Spanish Restaurant,Sporting Goods Shop,Sports Bar,Sports Club,Stadium,Steakhouse,Supermarket,Sushi Restaurant,Tapas Restaurant,Tea Room,Tennis Court,Tennis Stadium,Thai Restaurant,Theater,Theme Restaurant,Toy / Game Store,Trail,Train Station,Turkish Restaurant,Used Bookstore,Vegetarian / Vegan Restaurant,Venezuelan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Wine Shop,Women's Store,Yoga Studio,Zoo
count,63.000000,63.000000,63.000000,63.000000,63.000000,63.000000,63.000000,63.000000,63.000000,63.000000,63.000000,63.000000,63.000000,63.000000,63.000000,63.000000,63.000000,63.000000,63.000000,63.000000,63.000000,63.000000,63.000000,63.000000,63.000000,63.000000,63.000000,63.000000,63.000000,63.000000,63.000000,63.000000,63.000000,63.000000,63.000000,63.000000,63.000000,63.000000,63.000000,63.000000,63.000000,63.000000,63.000000,63.000000,63.000000,63.000000,63.000000,63.000000,63.000000,63.000000,63.000000,63.000000,63.000000,63.000000,63.000000,63.000000,63.000000,63.000000,63.000000,63.000000,63.000000,63.000000,63.000000,63.000000,63.000000,63.000000,63.000000,63.000000,63.000000,63.000000,63.000000,63.000000,63.000000,63.000000,63.000000,63.000000,63.000000,63.000000,63.000000,63.000000,63.000000,63.000000,63.000000,63.000000,63.000000,63.000000,63.0000

### Proportion of venues studied (restaurants, cultural spaces, hotels, green spots and shops) in each neighborhood

In [20]:
#for i in range(0,len(neighborhoods)):
    #map_list.append(grouped[neighborhoods[i]].sum(axis=1))


listaa=["American Restaurant","Argentinian Restaurant","Asian Restaurant","BBQ Joint","Brazilian Restaurant","Burrito Place","Chinese Restaurant","Diner","Dumpling Restaurant","Eastern European Restaurant","Falafel Restaurant","Fast Food Restaurant","Greek Restaurant","Indian Restaurant","Italian Restaurant","Japanese Restaurant","Kebab Restaurant","Korean Restaurant","Latin American Restaurant","Mediterranean Restaurant","Mexican Restaurant","Modern European Restaurant","Persian Restaurant","Peruvian Restaurant","Restaurant","Scandinavian Restaurant","Seafood Restaurant","Spanish Restaurant","Steakhouse","Sushi Restaurant","Tapas Restaurant","Thai Restaurant","Vegetarian / Vegan Restaurant","Vietnamese Restaurant","Burger Joint","Burrito Place","Cafeteria","Comfort Food Restaurant","Gastropub","North Indian Restaurant","Juice Bar","Paella Restaurant","Pizza Place","Polish Restaurant","Salad Place","Venezuelan Restaurant","Wine Bar","Tea Room","Art Gallery","Art Museum","Concert Hall","General Entertainment","History Museum","Library","Monument / Landmark","Museum","Movie Theater","Music Venue","Performing Arts Venue","Science Museum","Sculpture Garden","Stadium","Theater","Bed & Breakfast","Hostel","Hotel","Hotel Bar","Aquarium","Bridge","Fountain","Garden","Mountain","Park","Playground","Pool","Scenic Lookout","Sculpture Garden","Aquarium","Bridge","Fountain","Garden","Mountain","Park","Playground","Pool","Scenic Lookout","Sculpture Garden","Accessories Store","Bakery","Beer Store","Big Box Store","Boutique","Bubble Tea Shop","Candy Store","Cheese Shop","Chocolate Shop","Clothing Store","Coffee Shop","Convenience Store","Cosmetics Shop","Cupcake Shop","Dessert Shop","Discount Store","Donut Shop","Electronics Store","Fabric Shop","Food & Drink Shop","Furniture / Home Store","Gift Shop","Gourmet Shop","Grocery Store","Ice Cream Shop","Liquor Store","Miscellaneous Shop","Mobile Phone Shop","Motorcycle Shop","Pastry Shop","Pie Shop","Pet Store","Shopping Mall","Toy / Game Store","Wine Shop"]
grouped["total_suma"]=grouped[listaa].sum(axis=1)
grouped["total_suma"]

map_list = pd.DataFrame(neighborhoods, columns=['neighborhoods'])
map_list["number of venues"]=grouped["total_suma"]

map_list.head(15)

,neighborhoods,number of venues
0,Arganzuela,0.670000
1,Barajas,0.850000
2,Carabanchel,0.800000
3,Centro,0.820000
4,Chamartín,0.680000
5,Chamberí,0.710000
6,Ciudad Lineal,0.750000
7,Fuencarral - El Pardo,0.526316
8,Hortaleza,0.740000
9,Latina,0.750000


### Proportion of reastaurants in each neighborhood

In [21]:
listt_resm=["American Restaurant","Argentinian Restaurant","Asian Restaurant","BBQ Joint","Brazilian Restaurant","Burrito Place","Chinese Restaurant","Diner","Dumpling Restaurant","Eastern European Restaurant","Falafel Restaurant","Fast Food Restaurant","Greek Restaurant","Indian Restaurant","Italian Restaurant","Japanese Restaurant","Kebab Restaurant","Korean Restaurant","Latin American Restaurant","Mediterranean Restaurant","Mexican Restaurant","Modern European Restaurant","Persian Restaurant","Peruvian Restaurant","Restaurant","Scandinavian Restaurant","Seafood Restaurant","Spanish Restaurant","Steakhouse","Sushi Restaurant","Tapas Restaurant","Thai Restaurant","Vegetarian / Vegan Restaurant","Vietnamese Restaurant","Burger Joint","Burrito Place","Cafeteria","Comfort Food Restaurant","Gastropub","North Indian Restaurant","Juice Bar","Paella Restaurant","Pizza Place","Polish Restaurant","Salad Place","Venezuelan Restaurant","Wine Bar","Tea Room"]

grouped["total_sum_resm"]=grouped[listt_resm].sum(axis=1)

map_listt_resm = pd.DataFrame(neighborhoods, columns=['neighborhoods'])
map_listt_resm["number of venues"]=grouped["total_sum_resm"]

map_listt_resm.head(15)

,neighborhoods,number of venues
0,Arganzuela,0.310000
1,Barajas,0.370000
2,Carabanchel,0.530000
3,Centro,0.550000
4,Chamartín,0.370000
5,Chamberí,0.240000
6,Ciudad Lineal,0.250000
7,Fuencarral - El Pardo,0.289474
8,Hortaleza,0.500000
9,Latina,0.540000


### Proportion of cultural spaces in each neighborhood

In [22]:
listt_culm=["Art Gallery","Art Museum","Concert Hall","General Entertainment","History Museum","Library","Monument / Landmark","Museum","Movie Theater","Music Venue","Performing Arts Venue","Science Museum","Sculpture Garden","Stadium","Theater"]

grouped["total_sum_culm"]=grouped[listt_culm].sum(axis=1)

map_listt_culm = pd.DataFrame(neighborhoods, columns=['neighborhoods'])
map_listt_culm["number of venues"]=grouped["total_sum_culm"]

map_listt_culm.head(15)

,neighborhoods,number of venues
0,Arganzuela,0.110000
1,Barajas,0.110000
2,Carabanchel,0.060000
3,Centro,0.020000
4,Chamartín,0.080000
5,Chamberí,0.150000
6,Ciudad Lineal,0.160000
7,Fuencarral - El Pardo,0.000000
8,Hortaleza,0.040000
9,Latina,0.030000


### Proportion of hotels in each neighborhood

In [23]:
listt_hotm=["Bed & Breakfast","Hostel","Hotel","Hotel Bar"]

grouped["total_sum_hotm"]=grouped[listt_hotm].sum(axis=1)

map_listt_hotm = pd.DataFrame(neighborhoods, columns=['neighborhoods'])
map_listt_hotm["number of venues"]=grouped["total_sum_hotm"]

map_listt_hotm.head(15)

,neighborhoods,number of venues
0,Arganzuela,0.020000
1,Barajas,0.020000
2,Carabanchel,0.030000
3,Centro,0.060000
4,Chamartín,0.040000
5,Chamberí,0.030000
6,Ciudad Lineal,0.040000
7,Fuencarral - El Pardo,0.000000
8,Hortaleza,0.030000
9,Latina,0.050000


### Proportion of green spots in each neighborhood

In [24]:
listt_grem=["Aquarium","Bridge","Fountain","Garden","Mountain","Park","Playground","Pool","Scenic Lookout","Sculpture Garden"]

grouped["total_sum_grem"]=grouped[listt_grem].sum(axis=1)

map_listt_grem = pd.DataFrame(neighborhoods, columns=['neighborhoods'])
map_listt_grem["number of venues"]=grouped["total_sum_grem"]

map_listt_grem.head(15)

,neighborhoods,number of venues
0,Arganzuela,0.060000
1,Barajas,0.130000
2,Carabanchel,0.000000
3,Centro,0.030000
4,Chamartín,0.020000
5,Chamberí,0.080000
6,Ciudad Lineal,0.100000
7,Fuencarral - El Pardo,0.000000
8,Hortaleza,0.020000
9,Latina,0.010000


### Proportion of shops in each neighborhood

In [25]:
listt_shom=["Accessories Store","Bakery","Beer Store","Big Box Store","Boutique","Bubble Tea Shop","Candy Store","Cheese Shop","Chocolate Shop","Clothing Store","Coffee Shop","Convenience Store","Cosmetics Shop","Cupcake Shop","Dessert Shop","Discount Store","Donut Shop","Electronics Store","Fabric Shop","Food & Drink Shop","Furniture / Home Store","Gift Shop","Gourmet Shop","Grocery Store","Ice Cream Shop","Liquor Store","Miscellaneous Shop","Mobile Phone Shop","Motorcycle Shop","Pastry Shop","Pie Shop","Pet Store","Shopping Mall","Toy / Game Store","Wine Shop"]

grouped["total_sum_shom"]=grouped[listt_shom].sum(axis=1)

map_listt_shom = pd.DataFrame(neighborhoods, columns=['neighborhoods'])
map_listt_shom["number of venues"]=grouped["total_sum_shom"]

map_listt_shom.head(15)

,neighborhoods,number of venues
0,Arganzuela,0.110000
1,Barajas,0.090000
2,Carabanchel,0.180000
3,Centro,0.130000
4,Chamartín,0.150000
5,Chamberí,0.130000
6,Ciudad Lineal,0.100000
7,Fuencarral - El Pardo,0.236842
8,Hortaleza,0.130000
9,Latina,0.110000


## After analyzing Madrid's data, it is time to study the neighborhoods in Berlin to then compare both

### We scrap Berlin's neighborhoods data through Wikipedia

In [26]:
data_b = requests.get("https://en.wikipedia.org/wiki/Category:Localities_of_Berlin").text
soup_b = BeautifulSoup(data_b, 'lxml')
textList_b = []
neighborhoodList_b = []

In [27]:
# append the data into the list
neighborhoodList_b.clear()

for row in soup_b.find_all("div", class_="mw-category")[0].findAll("li"):
    neighborhoodList_b.append(row.text)
    
df_b = pd.DataFrame({"Neighborhood": neighborhoodList_b})
df1_b = df_b.iloc[1:]
berlin_df = df1_b.reset_index(drop=True)
berlin_df.head(15)

,Neighborhood
0,Adlershof
1,Afrikanisches Viertel
2,Alt-Hohenschönhausen
3,Alt-Treptow
4,Altglienicke
5,Baumschulenweg
6,Biesdorf (Berlin)
7,Blankenburg (Berlin)
8,Blankenfelde
9,Bohnsdorf


### We obtain the coordinates for each neighborhood in Berlin

In [28]:
# Geographical coordinates of neighborhoods

# define a function to get coordinates
def get_latlng(neighborhood):
    # initialize your variable to None
    lat_lng_coords = None
    # loop until you get the coordinates
    while(lat_lng_coords is None):
        g = geocoder.arcgis('{}, Berlin, Germany'.format(neighborhood))
        lat_lng_coords = g.latlng
    return lat_lng_coords

coords = [ get_latlng(neighborhood) for neighborhood in berlin_df["Neighborhood"].tolist() ]

df_coords = pd.DataFrame(coords, columns=['Latitude', 'Longitude'])

# merge the coordinates into the original dataframe
berlin_df['Latitude'] = df_coords['Latitude']
berlin_df['Longitude'] = df_coords['Longitude']


# check the neighborhoods and the coordinates
print(berlin_df.shape)
berlin_df.head(15)

(97, 3)


,Neighborhood,Latitude,Longitude
0,Adlershof,52.437790,13.54778
1,Afrikanisches Viertel,52.558269,13.33389
2,Alt-Hohenschönhausen,52.547060,13.50055
3,Alt-Treptow,52.493500,13.45711
4,Altglienicke,52.420060,13.53969
5,Baumschulenweg,52.466690,13.48840
6,Biesdorf (Berlin),52.510040,13.55374
7,Blankenburg (Berlin),52.593320,13.45805
8,Blankenfelde,52.618550,13.39017
9,Bohnsdorf,52.401090,13.56057


### We save the dataframe as a CSV file

In [29]:
berlin_df.to_csv("berlin_neighborhoods.csv", index=False)

### Obtain the exact coordinates of the city of Berlin

In [30]:
#address = 'Berlin, Germany'
#user_agent : geopy.geocoders.options.default_user_agent = "my-application"
#geolocator = Nominatim(user_agent = "my-application")
#location = geolocator.geocode(address)
#latitude = location.latitude
#longitude = location.longitude
#print('The geograpical coordinate of Berlin is {}, {}.'.format(latitude, longitude))
latitude=52.5170365
longitude=13.3888599
print('The geograpical coordinate of Berlin is 52.5170365, 13.3888599.')

The geograpical coordinate of Berlin is 52.5170365, 13.3888599.


### Create a map using the coordinate values (latitude and longitude) of each neighborhood

In [31]:
map_berlin = folium.Map(location=[latitude, longitude], zoom_start=11)

# add markers to map
for lat, lng, neighborhood in zip(berlin_df['Latitude'], berlin_df['Longitude'], berlin_df['Neighborhood']):
    label = '{}'.format(neighborhood)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7).add_to(map_berlin)  
    
map_berlin

### Like done before, we will use Foursquare to find the venues we want to study

In [32]:
CLIENT_ID = "ECUGWIPDIDJMU55URTU5W0BSUTVMVA0ZR4ZIYJVED0PUQKHM"
CLIENT_SECRET = "CWKKZIBB0UF15BVG2S1DEU43NS0K1L4ZV3QGYFLH0DLF44DR"
VERSION = "20210419"

In [33]:
radius = 2000
LIMIT = 100

venues = []
for lat, long, neighborhood in zip(berlin_df['Latitude'], berlin_df['Longitude'], berlin_df['Neighborhood']):
    
    # create the API request URL
    url = "https://api.foursquare.com/v2/venues/explore?client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}".format(
        CLIENT_ID,
        CLIENT_SECRET,
        VERSION,
        lat,
        long,
        radius, 
        LIMIT)
    
    # make the GET request
    results = requests.get(url).json()["response"]['groups'][0]['items']
    
    # return only relevant information for each nearby venue
    for venue in results:
        venues.append((
            neighborhood,
            lat, 
            long, 
            venue['venue']['name'], 
            venue['venue']['location']['lat'], 
            venue['venue']['location']['lng'],  
            venue['venue']['categories'][0]['name']))

In [34]:
# convert the venues list into a new DataFrame
venues_df = pd.DataFrame(venues)

# define the column names
venues_df.columns = ['Neighborhood', 'Latitude', 'Longitude', 'VenueName', 'VenueLatitude', 'VenueLongitude', 'VenueCategory']

print(venues_df.shape)
venues_df.head(15)

(5681, 7)


,Neighborhood,Latitude,Longitude,VenueName,VenueLatitude,VenueLongitude,VenueCategory
0,Adlershof,52.43779,13.54778,Mia Toscana,52.438327,13.549573,Italian Restaurant
1,Adlershof,52.43779,13.54778,Griechisches Restaurant Athen,52.434901,13.546680,Greek Restaurant
2,Adlershof,52.43779,13.54778,Olympia Greek Food,52.433982,13.538500,Greek Restaurant
3,Adlershof,52.43779,13.54778,McFIT,52.430956,13.549099,Gym / Fitness Center
4,Adlershof,52.43779,13.54778,Food - Taste of the world on the road,52.428786,13.538168,Food Truck
5,Adlershof,52.43779,13.54778,mani mogo,52.432893,13.531991,Korean Restaurant
6,Adlershof,52.43779,13.54778,Adapt Apartments Hotel,52.432655,13.532206,Hotel
7,Adlershof,52.43779,13.54778,Schloss Köpenick,52.443679,13.572549,Palace
8,Adlershof,52.43779,13.54778,dm-drogerie markt,52.437625,13.547692,Drugstore
9,Adlershof,52.43779,13.54778,Eisbar Friederici,52.444922,13.574279,Ice Cream Shop


In [35]:
venues_df.groupby(["Neighborhood"]).count()
print('There are {} uniques categories.'.format(len(venues_df['VenueCategory'].unique())))

There are 349 uniques categories.


### We study the proportion of each venue for each neighborhood in Berlin

In [36]:
# one hot encoding
onehot = pd.get_dummies(venues_df[['VenueCategory']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
onehot['Neighborhoods'] = venues_df['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [onehot.columns[-1]] + list(onehot.columns[:-1])
onehot = onehot[fixed_columns]

print(onehot.shape)

grouped = onehot.groupby(["Neighborhoods"]).mean().reset_index()

print(grouped.shape)
grouped.head(15)

(5681, 350)
(97, 350)


,Neighborhoods,ATM,Adult Boutique,African Restaurant,American Restaurant,Animal Shelter,Aquarium,Arcade,Argentinian Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Austrian Restaurant,Auto Dealership,Auto Garage,Auto Workshop,Automotive Shop,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Bathing Area,Beach,Beach Bar,Bed & Breakfast,Beer Bar,Beer Garden,Beer Store,Big Box Store,Bike Rental / Bike Share,Bike Shop,Bistro,Boarding House,Boat Rental,Boat or Ferry,Bookstore,Botanical Garden,Boutique,Bowling Alley,Bowling Green,Brasserie,Brazilian Restaurant,Breakfast Spot,Brewery,Bridge,Bubble Tea Shop,Building,Burger Joint,Burrito Place,Bus Stop,Business Service,Butcher,Cafeteria,Café,Cajun / Creole Restaurant,Campground,Canal,Canal Lock,Candy Store,Caribbean Restaurant,Castle,Caucasian Restaurant,Cemetery,Cheese Shop,Child Care Service,Chinese Restaurant,Chocolate Shop,Church,Circus,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Roaster,Coffee Shop,College Cafeteria,College Gym,College Rec Center,Comedy Club,Concert Hall,Convenience Store,Cosmetics Shop,Costume Shop,Credit Union,Creperie,Cultural Center,Cupcake Shop,Currywurst Joint,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Diner,Discount Store,Dive Bar,Dog Run,Doner Restaurant,Donut Shop,Drugstore,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Empanada Restaurant,Escape Room,Ethiopian Restaurant,Event Service,Event Space,Fabric Shop,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant,Field,Filipino Restaurant,Film Studio,Financial or Legal Service,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,Food & Drink Shop,Food Court,Food Stand,Food Truck,Football Stadium,Forest,Fountain,French Restaurant,Fried Chicken Joint,Furniture / Home Store,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,General Entertainment,German Restaurant,Gift Shop,Go Kart Track,Golf Course,Gourmet Shop,Greek Restaurant,Grocery Store,Gun Range,Gun Shop,Gym,Gym / Fitness Center,Halal Restaurant,Harbor / Marina,Hardware Store,Health & Beauty Service,Historic Site,History Museum,Hobby Shop,Hockey Field,Hockey Rink,Home Service,Hookah Bar,Hostel,Hot Dog Joint,Hotel,Hotel Bar,IT Services,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indie Theater,Indonesian Restaurant,Indoor Play Area,Insurance Office,Intersection,Irish Pub,Israeli Restaurant,Italian Restaurant,Japanese Restaurant,Jazz Club,Juice Bar,Kebab Restaurant,Korean Restaurant,Lake,Laser Tag,Latin American Restaurant,Laundromat,Laundry Service,Lebanese Restaurant,Library,Light Rail Station,Liquor Store,Lounge,Market,Massage Studio,Mediterranean Restaurant,Memorial Site,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Modern European Restaurant,Monument / Landmark,Motel,Motorcycle Shop,Mountain,Movie Theater,Multiplex,Museum,Music Venue,Nature Preserve,Neighborhood,New American Restaurant,Newsstand,Nightclub,Noodle House,Nudist Beach,Opera House,Optical Shop,Organic Grocery,Other Great Outdoors,Outdoor Sculpture,Outdoor Supply Store,Paintball Field,Pakistani Restaurant,Palace,Park,Pastry Shop,Pedestrian Plaza,Performing Arts Venue,Perfume Shop,Persian Restaurant,Peruvian Restaurant,Pet Café,Pet Service,Pet Store,Pharmacy,Photography Studio,Piano Bar,Pide Place,Pie Shop,Pier,Piercing Parlor,Pizza Place,Planetarium,Platform,Playground,Plaza,Pool,Pool Hall,Portuguese Restaurant,Post Office,Print Shop,Pub,Public Art,Racecourse,Ramen Restaurant,Record Shop,Recreation Center,Rental Car Location,Residential Building (Apartment / Condo),Restaurant,River,Road,Rock Climbing Spot,Rock Club,Roof Deck,Rooftop Bar,Rugby Stadium,Russian Restaurant,Salon / Barbershop,Sandwich Place,Sauna / Steam Room,Scandinavian Restaurant,Scenic Lookout,Schnitzel Restaurant,School,Science Museum,Sculpture Garden,Seafood Restaurant,Shawarma Place,Shipping Store,Shoe Store,Shopping Mall,Skate Park,Skating Rink,Smoke Sho

### Proportion of restaurants over the total number of venues for each neighborhood in Berlin

In [38]:
d=grouped["African Restaurant"]+grouped["American Restaurant"]+grouped["Argentinian Restaurant"]+grouped["Asian Restaurant"]+grouped["Austrian Restaurant"]+grouped["BBQ Joint"]+grouped["Brasserie"]+grouped["Brazilian Restaurant"]+grouped["Burrito Place"]+grouped["Cajun / Creole Restaurant"]+grouped["Caribbean Restaurant"]+grouped["Caucasian Restaurant"]+grouped["Chinese Restaurant"]+grouped["Diner"]+grouped["Doner Restaurant"]+grouped["Dumpling Restaurant"]+grouped["Eastern European Restaurant"]+grouped["Empanada Restaurant"]+grouped["Ethiopian Restaurant"]+grouped["Falafel Restaurant"]+grouped["Fast Food Restaurant"]+grouped["Filipino Restaurant"]+grouped["French Restaurant"]+grouped["Greek Restaurant"]+grouped["German Restaurant"]+grouped["Halal Restaurant"]+grouped["Indian Restaurant"]+grouped["Indonesian Restaurant"]+grouped["Israeli Restaurant"]+grouped["Italian Restaurant"]+grouped["Japanese Restaurant"]+grouped["Kebab Restaurant"]+grouped["Korean Restaurant"]+grouped["Latin American Restaurant"]+grouped["Lebanese Restaurant"]+grouped["Mediterranean Restaurant"]+grouped["Mexican Restaurant"]+grouped["Modern European Restaurant"]+grouped["New American Restaurant"]+grouped["Pakistani Restaurant"]+grouped["Persian Restaurant"]+grouped["Peruvian Restaurant"]+grouped["Portuguese Restaurant"]+grouped["Ramen Restaurant"]+grouped["Restaurant"]+grouped["Russian Restaurant"]+grouped["Scandinavian Restaurant"]+grouped["Schnitzel Restaurant"]+grouped["Seafood Restaurant"]+grouped["Spanish Restaurant"]+grouped["Steakhouse"]+grouped["Sushi Restaurant"]+grouped["Syrian Restaurant"]+grouped["Szechuan Restaurant"]+grouped["Tapas Restaurant"]+grouped["Thai Restaurant"]+grouped["Tibetan Restaurant"]+grouped["Turkish Home Cooking Restaurant"]+grouped["Vegetarian / Vegan Restaurant"]+grouped["Vietnamese Restaurant"]+grouped["Yemeni Restaurant"]
d.sum()

19.27622118695241

### Proportion of cultural spaces over the total number of venues for each neighborhood in Berlin

In [39]:
g=grouped["Art Gallery"]+grouped["Art Museum"]+grouped["Arts & Crafts Store"]+grouped["Concert Hall"]+grouped["General Entertainment"]+grouped["History Museum"]+grouped["Jazz Club"]+grouped["Library"]+grouped["Monument / Landmark"]+grouped["Museum"]+grouped["Movie Theater"]+grouped["Music Venue"]+grouped["Outdoor Sculpture"]+grouped["Palace"]+grouped["Performing Arts Venue"]+grouped["Science Museum"]+grouped["Sculpture Garden"]+grouped["Stadium"]+grouped["Theater"]
g.sum()

2.7624798317303503

### Proportion of hotels over the total number of venues for each neighborhood in Berlin

In [40]:
h=grouped["Bed & Breakfast"]+grouped["Hostel"]+grouped["Hotel"]+grouped["Hotel Bar"]+grouped["Motel"]
h.sum()

2.7249939898567255

### Proportion of green spots over the total number of venues for each neighborhood in Berlin

In [41]:
j=grouped["Aquarium"]+grouped["Beach"]+grouped["Botanical Garden"]+grouped["Bridge"]+grouped["Campground"]+grouped["Canal"]+grouped["Canal Lock"]+grouped["Cemetery"]+grouped["Farm"]+grouped["Field"]+grouped["Forest"]+grouped["Fountain"]+grouped["Garden"]+grouped["Harbor / Marina"]+grouped["Mountain"]+grouped["Nature Preserve"]+grouped["Other Great Outdoors"]+grouped["Park"]+grouped["Pier"]+grouped["Playground"]+grouped["River"]+grouped["Pool"]+grouped["Scenic Lookout"]+grouped["Sculpture Garden"]+grouped["Tree"]+grouped["Waterfall"]+grouped["Windmill"]
j.sum()

8.176948546146951

### Proportion of shops over the total number of venues for each neighborhood in Berlin

In [43]:
k=grouped["Arts & Crafts Store"]+grouped["Automotive Shop"]+grouped["Baby Store"]+grouped["Bagel Shop"]+grouped["Bakery"]+grouped["Beer Store"]+grouped["Big Box Store"]+grouped["Bike Shop"]+grouped["Boutique"]+grouped["Bubble Tea Shop"]+grouped["Candy Store"]+grouped["Cheese Shop"]+grouped["Chocolate Shop"]+grouped["Clothing Store"]+grouped["Coffee Shop"]+grouped["Convenience Store"]+grouped["Cosmetics Shop"]+grouped["Costume Shop"]+grouped["Cupcake Shop"]+grouped["Dessert Shop"]+grouped["Discount Store"]+grouped["Donut Shop"]+grouped["Electronics Store"]+grouped["Fabric Shop"]+grouped["Fish & Chips Shop"]+grouped["Flower Shop"]+grouped["Food & Drink Shop"]+grouped["Furniture / Home Store"]+grouped["Gift Shop"]+grouped["Gourmet Shop"]+grouped["Grocery Store"]+grouped["Hardware Store"]+grouped["Hobby Shop"]+grouped["Ice Cream Shop"]+grouped["Liquor Store"]+grouped["Men's Store"]+grouped["Miscellaneous Shop"]+grouped["Mobile Phone Shop"]+grouped["Motorcycle Shop"]+grouped["Pastry Shop"]+grouped["Pie Shop"]+grouped["Pet Store"]+grouped["Record Shop"]+grouped["Shipping Store"]+grouped["Shoe Store"]+grouped["Shopping Mall"]+grouped["Smoke Shop"]+grouped["Toy / Game Store"]+grouped["Wine Shop"]+grouped["Video Store"]
k.sum()

13.338501516654663

### General description of the results obatined

In [44]:
grouped.describe()

,ATM,Adult Boutique,African Restaurant,American Restaurant,Animal Shelter,Aquarium,Arcade,Argentinian Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Austrian Restaurant,Auto Dealership,Auto Garage,Auto Workshop,Automotive Shop,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Bathing Area,Beach,Beach Bar,Bed & Breakfast,Beer Bar,Beer Garden,Beer Store,Big Box Store,Bike Rental / Bike Share,Bike Shop,Bistro,Boarding House,Boat Rental,Boat or Ferry,Bookstore,Botanical Garden,Boutique,Bowling Alley,Bowling Green,Brasserie,Brazilian Restaurant,Breakfast Spot,Brewery,Bridge,Bubble Tea Shop,Building,Burger Joint,Burrito Place,Bus Stop,Business Service,Butcher,Cafeteria,Café,Cajun / Creole Restaurant,Campground,Canal,Canal Lock,Candy Store,Caribbean Restaurant,Castle,Caucasian Restaurant,Cemetery,Cheese Shop,Child Care Service,Chinese Restaurant,Chocolate Shop,Church,Circus,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Roaster,Coffee Shop,College Cafeteria,College Gym,College Rec Center,Comedy Club,Concert Hall,Convenience Store,Cosmetics Shop,Costume Shop,Credit Union,Creperie,Cultural Center,Cupcake Shop,Currywurst Joint,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Diner,Discount Store,Dive Bar,Dog Run,Doner Restaurant,Donut Shop,Drugstore,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Empanada Restaurant,Escape Room,Ethiopian Restaurant,Event Service,Event Space,Fabric Shop,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant,Field,Filipino Restaurant,Film Studio,Financial or Legal Service,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,Food & Drink Shop,Food Court,Food Stand,Food Truck,Football Stadium,Forest,Fountain,French Restaurant,Fried Chicken Joint,Furniture / Home Store,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,General Entertainment,German Restaurant,Gift Shop,Go Kart Track,Golf Course,Gourmet Shop,Greek Restaurant,Grocery Store,Gun Range,Gun Shop,Gym,Gym / Fitness Center,Halal Restaurant,Harbor / Marina,Hardware Store,Health & Beauty Service,Historic Site,History Museum,Hobby Shop,Hockey Field,Hockey Rink,Home Service,Hookah Bar,Hostel,Hot Dog Joint,Hotel,Hotel Bar,IT Services,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indie Theater,Indonesian Restaurant,Indoor Play Area,Insurance Office,Intersection,Irish Pub,Israeli Restaurant,Italian Restaurant,Japanese Restaurant,Jazz Club,Juice Bar,Kebab Restaurant,Korean Restaurant,Lake,Laser Tag,Latin American Restaurant,Laundromat,Laundry Service,Lebanese Restaurant,Library,Light Rail Station,Liquor Store,Lounge,Market,Massage Studio,Mediterranean Restaurant,Memorial Site,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Modern European Restaurant,Monument / Landmark,Motel,Motorcycle Shop,Mountain,Movie Theater,Multiplex,Museum,Music Venue,Nature Preserve,Neighborhood,New American Restaurant,Newsstand,Nightclub,Noodle House,Nudist Beach,Opera House,Optical Shop,Organic Grocery,Other Great Outdoors,Outdoor Sculpture,Outdoor Supply Store,Paintball Field,Pakistani Restaurant,Palace,Park,Pastry Shop,Pedestrian Plaza,Performing Arts Venue,Perfume Shop,Persian Restaurant,Peruvian Restaurant,Pet Café,Pet Service,Pet Store,Pharmacy,Photography Studio,Piano Bar,Pide Place,Pie Shop,Pier,Piercing Parlor,Pizza Place,Planetarium,Platform,Playground,Plaza,Pool,Pool Hall,Portuguese Restaurant,Post Office,Print Shop,Pub,Public Art,Racecourse,Ramen Restaurant,Record Shop,Recreation Center,Rental Car Location,Residential Building (Apartment / Condo),Restaurant,River,Road,Rock Climbing Spot,Rock Club,Roof Deck,Rooftop Bar,Rugby Stadium,Russian Restaurant,Salon / Barbershop,Sandwich Place,Sauna / Steam Room,Scandinavian Restaurant,Scenic Lookout,Schnitzel Restaurant,School,Science Museum,Sculpture Garden,Seafood Restaurant,Shawarma Place,Shipping Store,Shoe Store,Shopping Mall,Skate Park,Skating Rink,Smoke Shop,Snack Place,

### Proportion of the venues studied (restaurants, cultural spaces, hotels, green spots and shops) for each neighborhood in Berlin

In [46]:
#for i in range(0,len(neighborhoods)):
    #map_list.append(grouped[neighborhoods[i]].sum(axis=1))


listt=["African Restaurant","American Restaurant","Argentinian Restaurant","Asian Restaurant","Austrian Restaurant","BBQ Joint","Brasserie","Brazilian Restaurant","Burrito Place","Cajun / Creole Restaurant","Caribbean Restaurant","Caucasian Restaurant","Chinese Restaurant","Diner","Doner Restaurant","Dumpling Restaurant","Eastern European Restaurant","Empanada Restaurant","Ethiopian Restaurant","Falafel Restaurant","Fast Food Restaurant","Filipino Restaurant","French Restaurant","Greek Restaurant","German Restaurant","Halal Restaurant","Indian Restaurant","Indonesian Restaurant","Israeli Restaurant","Italian Restaurant","Japanese Restaurant","Kebab Restaurant","Korean Restaurant","Latin American Restaurant","Lebanese Restaurant","Mediterranean Restaurant","Mexican Restaurant","Modern European Restaurant","New American Restaurant","Pakistani Restaurant","Persian Restaurant","Peruvian Restaurant","Portuguese Restaurant","Ramen Restaurant","Restaurant","Russian Restaurant","Scandinavian Restaurant","Schnitzel Restaurant","Seafood Restaurant","Spanish Restaurant","Steakhouse","Sushi Restaurant","Syrian Restaurant","Szechuan Restaurant","Tapas Restaurant","Thai Restaurant","Tibetan Restaurant","Turkish Home Cooking Restaurant","Vegetarian / Vegan Restaurant","Vietnamese Restaurant","Yemeni Restaurant","Art Gallery","Art Museum","Arts & Crafts Store","Concert Hall","General Entertainment","History Museum","Jazz Club","Library","Monument / Landmark","Museum","Movie Theater","Music Venue","Outdoor Sculpture","Palace","Performing Arts Venue","Science Museum","Sculpture Garden","Stadium","Theater","Bed & Breakfast","Hostel","Hotel","Hotel Bar","Motel","Aquarium","Beach","Botanical Garden","Bridge","Campground","Canal","Canal Lock","Cemetery","Farm","Field","Forest","Fountain","Garden","Harbor / Marina","Mountain","Nature Preserve","Other Great Outdoors","Park","Pier","Playground","River","Pool","Scenic Lookout","Sculpture Garden","Tree","Waterfall","Windmill","Arts & Crafts Store","Automotive Shop","Baby Store","Bagel Shop","Bakery","Beer Store","Big Box Store","Bike Shop","Boutique","Bubble Tea Shop","Candy Store","Cheese Shop","Chocolate Shop","Clothing Store","Coffee Shop","Convenience Store","Cosmetics Shop","Costume Shop","Cupcake Shop","Dessert Shop","Discount Store","Donut Shop","Electronics Store","Fabric Shop","Fish & Chips Shop","Flower Shop","Food & Drink Shop","Furniture / Home Store","Gift Shop","Gourmet Shop","Grocery Store","Hardware Store","Hobby Shop","Ice Cream Shop","Liquor Store","Men's Store","Miscellaneous Shop","Mobile Phone Shop","Motorcycle Shop","Pastry Shop","Pie Shop","Pet Store","Record Shop","Shipping Store","Shoe Store","Shopping Mall","Smoke Shop","Toy / Game Store","Wine Shop","Video Store"]
grouped["total_sum"]=grouped[listt].sum(axis=1)
grouped["total_sum"].head(15)

0     0.500000
1     0.478723
2     0.373134
3     0.520000
4     0.466667
5     0.412698
6     0.387097
7     0.416667
8     0.555556
9     0.526316
10    0.531915
11    0.564103
12    0.333333
13    0.568182
14    0.640000
Name: total_sum, dtype: float64

In [54]:
map_listt = pd.DataFrame(neighborhoodList_b[1:], columns=['neighborhoods'])
map_listt["number of venues"]=grouped["total_sum"]

map_listt.head(15)

,neighborhoods,number of venues
0,Adlershof,0.500000
1,Afrikanisches Viertel,0.478723
2,Alt-Hohenschönhausen,0.373134
3,Alt-Treptow,0.520000
4,Altglienicke,0.466667
5,Baumschulenweg,0.412698
6,Biesdorf (Berlin),0.387097
7,Blankenburg (Berlin),0.416667
8,Blankenfelde,0.555556
9,Bohnsdorf,0.526316


### Proportion of restaurants for each neighborhood in Berlin

In [53]:
listt_res=["African Restaurant","American Restaurant","Argentinian Restaurant","Asian Restaurant","Austrian Restaurant","BBQ Joint","Brasserie","Brazilian Restaurant","Burrito Place","Cajun / Creole Restaurant","Caribbean Restaurant","Caucasian Restaurant","Chinese Restaurant","Diner","Doner Restaurant","Dumpling Restaurant","Eastern European Restaurant","Empanada Restaurant","Ethiopian Restaurant","Falafel Restaurant","Fast Food Restaurant","Filipino Restaurant","French Restaurant","Greek Restaurant","German Restaurant","Halal Restaurant","Indian Restaurant","Indonesian Restaurant","Israeli Restaurant","Italian Restaurant","Japanese Restaurant","Kebab Restaurant","Korean Restaurant","Latin American Restaurant","Lebanese Restaurant","Mediterranean Restaurant","Mexican Restaurant","Modern European Restaurant","New American Restaurant","Pakistani Restaurant","Persian Restaurant","Peruvian Restaurant","Portuguese Restaurant","Ramen Restaurant","Restaurant","Russian Restaurant","Scandinavian Restaurant","Schnitzel Restaurant","Seafood Restaurant","Spanish Restaurant","Steakhouse","Sushi Restaurant","Syrian Restaurant","Szechuan Restaurant","Tapas Restaurant","Thai Restaurant","Tibetan Restaurant","Turkish Home Cooking Restaurant","Vegetarian / Vegan Restaurant","Vietnamese Restaurant","Yemeni Restaurant"]
grouped["total_sum_res"]=grouped[listt_res].sum(axis=1)

map_listt_res = pd.DataFrame(neighborhoodList_b[1:], columns=['neighborhoods'])
map_listt_res["number of venues"]=grouped["total_sum_res"]

map_listt_res.head(15)

,neighborhoods,number of venues
0,Adlershof,0.270833
1,Afrikanisches Viertel,0.223404
2,Alt-Hohenschönhausen,0.119403
3,Alt-Treptow,0.200000
4,Altglienicke,0.200000
5,Baumschulenweg,0.142857
6,Biesdorf (Berlin),0.129032
7,Blankenburg (Berlin),0.250000
8,Blankenfelde,0.000000
9,Bohnsdorf,0.105263


### Proportion of cultural spaces for each neighborhood in Berlin

In [56]:
listt_cul=["Art Gallery","Art Museum","Arts & Crafts Store","Concert Hall","General Entertainment","History Museum","Jazz Club","Library","Monument / Landmark","Museum","Movie Theater","Music Venue","Outdoor Sculpture","Palace","Performing Arts Venue","Science Museum","Sculpture Garden","Stadium","Theater"]

grouped["total_sum_cul"]=grouped[listt_cul].sum(axis=1)

map_listt_cul = pd.DataFrame(neighborhoodList_b[1:], columns=['neighborhoods'])
map_listt_cul["number of venues"]=grouped["total_sum_cul"]

map_listt_cul.head(15)

,neighborhoods,number of venues
0,Adlershof,0.020833
1,Afrikanisches Viertel,0.031915
2,Alt-Hohenschönhausen,0.014925
3,Alt-Treptow,0.010000
4,Altglienicke,0.033333
5,Baumschulenweg,0.031746
6,Biesdorf (Berlin),0.032258
7,Blankenburg (Berlin),0.000000
8,Blankenfelde,0.000000
9,Bohnsdorf,0.052632


### Proportion of hotels for each neighborhood in Berlin

In [57]:
listt_hot=["Bed & Breakfast","Hostel","Hotel","Hotel Bar","Motel"]

grouped["total_sum_hot"]=grouped[listt_hot].sum(axis=1)

map_listt_hot = pd.DataFrame(neighborhoodList_b[1:], columns=['neighborhoods'])
map_listt_hot["number of venues"]=grouped["total_sum_hot"]

map_listt_hot.head(15)

,neighborhoods,number of venues
0,Adlershof,0.083333
1,Afrikanisches Viertel,0.010638
2,Alt-Hohenschönhausen,0.044776
3,Alt-Treptow,0.050000
4,Altglienicke,0.066667
5,Baumschulenweg,0.015873
6,Biesdorf (Berlin),0.032258
7,Blankenburg (Berlin),0.083333
8,Blankenfelde,0.000000
9,Bohnsdorf,0.157895


### Proportion of green spots for each neighborhood in Berlin

In [58]:
listt_gre=["Aquarium","Beach","Botanical Garden","Bridge","Campground","Canal","Canal Lock","Cemetery","Farm","Field","Forest","Fountain","Garden","Harbor / Marina","Mountain","Nature Preserve","Other Great Outdoors","Park","Pier","Playground","River","Pool","Scenic Lookout","Sculpture Garden","Tree","Waterfall","Windmill"]

grouped["total_sum_gre"]=grouped[listt_gre].sum(axis=1)

map_listt_gre = pd.DataFrame(neighborhoodList_b[1:], columns=['neighborhoods'])
map_listt_gre["number of venues"]=grouped["total_sum_gre"]

map_listt_gre.head(15)

,neighborhoods,number of venues
0,Adlershof,0.020833
1,Afrikanisches Viertel,0.095745
2,Alt-Hohenschönhausen,0.014925
3,Alt-Treptow,0.050000
4,Altglienicke,0.033333
5,Baumschulenweg,0.079365
6,Biesdorf (Berlin),0.064516
7,Blankenburg (Berlin),0.000000
8,Blankenfelde,0.444444
9,Bohnsdorf,0.000000


### Proportion of shops for each neighborhood in Berlin

In [60]:
listt_sho=["Arts & Crafts Store","Automotive Shop","Baby Store","Bagel Shop","Bakery","Beer Store","Big Box Store","Bike Shop","Boutique","Bubble Tea Shop","Candy Store","Cheese Shop","Chocolate Shop","Clothing Store","Coffee Shop","Convenience Store","Cosmetics Shop","Costume Shop","Cupcake Shop","Dessert Shop","Discount Store","Donut Shop","Electronics Store","Fabric Shop","Fish & Chips Shop","Flower Shop","Food & Drink Shop","Furniture / Home Store","Gift Shop","Gourmet Shop","Grocery Store","Hardware Store","Hobby Shop","Ice Cream Shop","Liquor Store","Men's Store","Miscellaneous Shop","Mobile Phone Shop","Motorcycle Shop","Pastry Shop","Pie Shop","Pet Store","Record Shop","Shipping Store","Shoe Store","Shopping Mall","Smoke Shop","Toy / Game Store","Wine Shop","Video Store"]

grouped["total_sum_sho"]=grouped[listt_sho].sum(axis=1)

map_listt_sho = pd.DataFrame(neighborhoodList_b[1:], columns=['neighborhoods'])
map_listt_sho["number of venues"]=grouped["total_sum_sho"]

map_listt_sho.head(15)

,neighborhoods,number of venues
0,Adlershof,0.104167
1,Afrikanisches Viertel,0.117021
2,Alt-Hohenschönhausen,0.179104
3,Alt-Treptow,0.210000
4,Altglienicke,0.133333
5,Baumschulenweg,0.142857
6,Biesdorf (Berlin),0.129032
7,Blankenburg (Berlin),0.083333
8,Blankenfelde,0.111111
9,Bohnsdorf,0.210526
